If you're not running in Saturn Cloud, you need to install these libraries:

Make sure you use the latest versions

```
pip install -U transformers accelerate bitsandbytes
```

In [ ]:
# In this machine in Saturn Cloud I don't have enough space in my home directory to run the model. I can see the space with the next command
#!df -l

# But I can see that under the directory "/run" I have enough space. So I'll provide an environment variable called "HF_HOME" with the route of this directory
# so that the model files could be downloaded inside it, where I have space.
import os
os.environ['HF_HOME'] = '/run/cache/'

In [1]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-13 12:33:48--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: 'minsearch.py'

     0K ...                                                   100%  969K=0.004s

2024-06-13 12:33:49 (969 KB/s) - 'minsearch.py' saved [3832/3832]



In [2]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [3]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [5]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
# pip install accelerate
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
# Tokenizer: turns text into a representation (numeric vector) that the model/llm can understand.
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")

# Actual Model/LLM I use.
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", device_map="auto")

In [ ]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

Explanation of Parameters for next method:

- **max_length**: Set this to a higher value if you want longer responses. For example, max_length=300.
- **num_beams**: Increasing this can lead to more thorough exploration of possible sequences. Typical values are between 5 and 10.
- **do_sample**: Set this to True to use sampling methods. This can produce more diverse responses.
- **temperature**: Lowering this value makes the model more confident and deterministic, while higher values increase diversity. Typical values range from 0.7 to 1.5.
- **top_k** and **top_p**: These parameters control nucleus sampling. top_k limits the sampling pool to the top k tokens, while top_p uses cumulative probability to cut off the sampling pool. Adjust these based on the desired level of randomness.


In [ ]:
def llm(prompt, generate_params=None):
    if generate_params is None:
        generate_params = {}

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(
        input_ids,
        max_length=generate_params.get("max_length", 100),
        num_beams=generate_params.get("num_beams", 5),
        do_sample=generate_params.get("do_sample", False),
        temperature=generate_params.get("temperature", 1.0),
        top_k=generate_params.get("top_k", 50),
        top_p=generate_params.get("top_p", 0.95),
    )
    result = tokenizer.decode(outputs[0])

    return result

In [ ]:
rag("I just discovered the course. Can I still join it?")